In [5]:
import os
import yaml
import pyEXP

# I have the Better run here; obviously another use will want to
# change this a directory containing some snapshots of their own
#
os.chdir('/data/Nbody/Better')

## Create the basis
We'll only do the halo coefficients in this simple example.  The cylindrical coefficients would procede similarly.  See the `sample_basis` notebook for an example of creating the cylindrical basis.

In [6]:
# Get the basis config
#
yaml_config = ""
#with open('basis.yaml') as f:
#    config = yaml.load(f, Loader=yaml.FullLoader)
#    yaml_config = yaml.dump(config)

# Alternatively, you could construct this on the fly, e.g.
bconfig = """
---
id: sphereSL
parameters :
  numr: 2000
  rmin: 0.0001
  rmax: 1.95
  Lmax: 4
  nmax: 10
  scale: 0.0667
  modelname: SLGridSph.model
...
"""
print('-'*60)
print('Read from file')
print('-'*60)
print(yaml_config)
print('-'*60)
print('Constructed from string')
print('-'*60)
print(bconfig)
print('-'*60)

yaml_config = bconfig

# Construct the basis instance
#
basis   = pyEXP.basis.Basis.factory(yaml_config)

------------------------------------------------------------
Read from file
------------------------------------------------------------

------------------------------------------------------------
Constructed from string
------------------------------------------------------------

---
id: sphereSL
parameters :
  numr: 2000
  rmin: 0.0001
  rmax: 1.95
  Lmax: 4
  nmax: 10
  scale: 0.0667
  modelname: SLGridSph.model
...

------------------------------------------------------------
---- SLGridSph::ReadH5Cache: parameter scale: wanted 0.0667 found 1
---- SLGridSph::WriteH5Cache: wrote <.slgrid_sph_cache>


# An example of creating coefficients using pyEXP
This example assumes that you have run the `EXP/examples/Better` simulation.  But this notebook could be adapted for any simulation you like.

We begin by importing `pyEXP` and friends and setting the working directory.

## Creating a particle reader
Now that we have a basis, we can use it to create coefficients from the particle snapshots.  `pyEXP` uses a `ParticleReader` object for that.

The first step is to hand off the files that comprise a snapshot for every time slice.  The `ParticleReader` provides a helper function for that.   There are two helper functions: `parseFileList` and `parseStringList`.  The first reads a list from a file and the second takes a list.  Otherwise they are the same.  The file names in the list are assumed to end with a snapshot index and an optional part index.  For example, if you have single files per snapshot, the list might look like: `myrun.00000`, `myrun.00001`, etc.  If you have multiple files per snapshot, they will look something like `myrun.00000_0001`, `myrun.00000_0002`, `myrun.00001_0000`, `myrun.00001_0001`, etc.

Here is the call for a file:

In [7]:
# Construct batches of files the particle reader.  One could use the
# parseStringList to create batches from a vector/list of files.  NB:
# a std::vector in C++ becomes a Python.list and vice versa
#
batches = pyEXP.read.ParticleReader.parseFileList('file.list', '')

We now iterate the `batches` created by the file parser to create the coefficients.   For each batch we create a new reader and pass the reader to the basis instance.  The `basis.createFromReader` member creates and returns the coefficients.  The coefficients are added to a coefficient container called `coefs`.  Note: on the first call `coefs=None` so a new container is created on the first time through.

In [8]:
# This will contain the coefficient container, need to start will a
# null instance to trigger construction
#
coefs   = None

for group in batches:

    print("file group is", group)

    # Make the reader for the desired type.  One could probably try to
    # do this by inspection but that's another project.
    #
    reader = pyEXP.read.ParticleReader.createReader('PSPout', group, 0, False);

    # Print the type list
    #
    print('The component names are:', reader.GetTypes())

    compname = 'dark halo'
    reader.SelectType(compname)
    print('Selected', compname)

    print('Call createFromReader at Time', reader.CurrentTime(), 'for', reader.CurrentNumber(), 'particles')

    coef = basis.createFromReader(reader)
    print("Created coef")

    # We need this stupid idiom here because None is not mapping to a
    # null pointer.  There is probably a way to do this.  Suggestions
    # anyone?
    #                          This is optional---+
    #                                             |
    if coefs is None:           #                 v
        coefs = pyEXP.coefs.Coefs.makecoefs(coef, compname)
    else:
        coefs.add(coef)

    print('Added coef')
    print('-'*60)

print('\nCompleted the file group list\n')

print('The coefficient time list is', coefs.Times())

file group is ['OUT.run0.00000']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.0 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00001']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.009999999999999764 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00002']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.019999999999998665 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00003']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.029999999999997563 for 100000 particles
Created coef
Added coef
----------------------------------

Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00035']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.23000000000010154 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00036']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.24000000000011376 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00037']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.25000000000012595 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00038']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0

Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00068']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.5599999999998297 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00069']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.5699999999998775 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00070']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.5799999999999252 for 100000 particles
Created coef
Added coef
------------------------------------------------------------
file group is ['OUT.run0.00071']
The component names are: ['dark halo', 'star disk']
Selected dark halo
Call createFromReader at Time 0.58

## Using a FieldGenerator
Now that we have our new coefficients, we can use the `FieldGenerator` to view the BFE representation of the underlying fields.  Here is an example:

In [9]:
# Now try some slices for rendering
#

times = coefs.Times()[0:3]
pmin  = [-1.0, -1.0, 0.0]
pmax  = [ 1.0,  1.0, 0.0]
grid  = [  40,   40,   0]

fields = pyEXP.field.FieldGenerator(times, pmin, pmax, grid)

surfaces = fields.slices(basis, coefs)

print("We now have the following [time field] pairs")
for v in surfaces:
    print('-'*40)
    for u in surfaces[v]:
        print("{:8.4f}  {}".format(v, u))

print("\nHere is the first one:")
for v in surfaces:
    for u in surfaces[v]:
        print('-'*40)
        print('----', u)
        print('-'*40)
        print(surfaces[v][u])
    break

We now have the following [time field] pairs
----------------------------------------
  0.0100  d
  0.0100  d0
  0.0100  d1
  0.0100  dd
  0.0100  fp
  0.0100  fr
  0.0100  ft
  0.0100  p
  0.0100  p0
  0.0100  p1
----------------------------------------
  0.0200  d
  0.0200  d0
  0.0200  d1
  0.0200  dd
  0.0200  fp
  0.0200  fr
  0.0200  ft
  0.0200  p
  0.0200  p0
  0.0200  p1
----------------------------------------
  0.0300  d
  0.0300  d0
  0.0300  d1
  0.0300  dd
  0.0300  fp
  0.0300  fr
  0.0300  ft
  0.0300  p
  0.0300  p0
  0.0300  p1

Here is the first one:
----------------------------------------
---- d
----------------------------------------
[[0.01483492 0.01596036 0.01714863 ... 0.01672036 0.01557327 0.01447744]
 [0.016003   0.01726649 0.01860924 ... 0.01800256 0.01671685 0.01549689]
 [0.01722879 0.01864417 0.02016226 ... 0.01938811 0.017938   0.01657651]
 ...
 [0.01678771 0.01815227 0.01959217 ... 0.0202994  0.01882689 0.01744738]
 [0.01564934 0.01686422 0.01813212 ...

These could be make into images and so forth.   We'll do this in another example notebook.

## Saving the coefficients

At this point, it makes sense to save the coefficients that you have just created.  This is sone with the following call:

In [10]:
coefs.WriteH5Coefs('test_better')

We now have a EXP HDF5 coefficient file called `test_better.h5`.    As an example, let's try reading the newly created file into another coefficient container, `coefs2`.  The container has a member function called `CompareStanzas` which will check on the contents.  Let's do it.

In [11]:
# Now try reading it in
#
coefs2 = pyEXP.coefs.Coefs.factory('test_better.h5')
print("Type is", coefs2.getGeometry())

# Now compare with the original
#
coefs2.CompareStanzas(coefs)

Type is sphere


False

Can't find Time=0.69
Can't find Time=0.7
Can't find Time=0.71
Can't find Time=0.72
Can't find Time=0.73
Can't find Time=0.74
Can't find Time=0.75
Can't find Time=0.76
Can't find Time=0.77
Can't find Time=0.78
Can't find Time=0.79
Can't find Time=0.8
Times in other coeffcients are: 0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2 0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28 0.29 0.3 0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4 0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5 0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6 0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68


This member function will print differences.  No differenced should be printed, of course.

In [17]:
cx = coefs2.getCoefStruct(0.68)
cc = SphCoefs()

NameError: name 'SphCoefs' is not defined